# Consumption Layer

In this notebook, we make the reference tables that will be consumed. These tables will hold clean reference data, as well as a change history of this data. For each table from the staging layer, we add three new fields to the change history:
*   `effective_time<TIMESTAMP>` to indicate the time that the record became active in the table.
*   `discontinue_time<TIMESTAMP>` to indicate the time that the record became inactive in the table.
*   `active_flag<BOOL>` whether the record is active or not.

Note that we don't technically need `active_flag`. We can infer it from the `discontinue_time` value. However, it is convenient to have this field for querying and reporting.


In [ ]:
%%bigquery
select table_name from airline_stg.INFORMATION_SCHEMA.TABLES order by table_name

Query is running:   0%|          |

Downloading:   0%|          |

,table_name
0,Air_Carrier
1,Airport
2,Flight
3,Flight_History
4,Meal
5,Meal_Service
6,Snack
7,Snack_Service


# Air_Carrier

In [ ]:
%%bigquery
create or replace table airline_csp.Air_Carrier
  as select *, current_timestamp() as effective_time, null as discontinue_time, true as active_flag
  from airline_stg.Air_Carrier

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select * from airline_csp.Air_Carrier
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,airline_id,airline_name,airline_code,data_source,load_time,effective_time,discontinue_time,active_flag
0,19107,Air Oregon,ARO,bird,2024-01-26 22:23:22.051288+00:00,2024-02-11 21:38:50.116391+00:00,<NA>,True
1,19244,Marshall's Air Inc.,MAR,bird,2024-01-26 22:23:22.051288+00:00,2024-02-11 21:38:50.116391+00:00,<NA>,True
2,19815,Air South,ASH,bird,2024-01-26 22:23:22.051288+00:00,2024-02-11 21:38:50.116391+00:00,<NA>,True
3,20142,Flagship Airlines Inc.,8N,bird,2024-01-26 22:23:22.051288+00:00,2024-02-11 21:38:50.116391+00:00,<NA>,True
4,19946,Great Western Aviation,GWA,bird,2024-01-26 22:23:22.051288+00:00,2024-02-11 21:38:50.116391+00:00,<NA>,True


In [ ]:
%%bigquery
alter table airline_csp.Air_Carrier add primary key (airline_id, effective_time) not enforced

Query is running:   0%|          |

""


# Airport

In [ ]:
%%bigquery
create or replace table airline_csp.Airport
  as select *, current_timestamp() as effective_time, null as discontinue_time, true as active_flag
  from airline_stg.Airport

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select * from airline_csp.Airport
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,iata,icao,name,city,state,country,data_source,load_time,effective_time,discontinue_time,active_flag
0,SRH,None,Sarh Airport,Sarh,None,Chad,bird_faker,2024-01-26 22:23:29.069725+00:00,2024-02-11 21:39:12.182922+00:00,<NA>,True
1,NDJ,None,Hassan Djamous,Ndjamena,None,Chad,bird_faker,2024-01-26 22:23:29.069725+00:00,2024-02-11 21:39:12.182922+00:00,<NA>,True
2,AMO,None,Mao Airport,Mao,None,Chad,bird_faker,2024-01-26 22:23:29.069725+00:00,2024-02-11 21:39:12.182922+00:00,<NA>,True
3,MQQ,None,Moundou Airport,Moundou,None,Chad,bird_faker,2024-01-26 22:23:29.069725+00:00,2024-02-11 21:39:12.182922+00:00,<NA>,True
4,AEH,None,Abeche Airport,Abeche,None,Chad,bird_faker,2024-01-26 22:23:29.069725+00:00,2024-02-11 21:39:12.182922+00:00,<NA>,True


In [ ]:
%%bigquery
alter table airline_csp.Airport add primary key (iata, effective_time) not enforced

Query is running:   0%|          |

""


# Flight

In [ ]:
%%bigquery
create or replace table airline_csp.Flight
  as select *, current_timestamp() as effective_time, null as discontinue_time, true as active_flag
  from airline_stg.Flight

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select * from airline_csp.Flight
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,fl_num,op_carrier_airline_id,op_carrier_fl_num,origin_airport_id,origin_airport_seq_id,origin_city_market_id,origin,dest_airport_id,dest_airport_seq_id,dest_city_market_id,dest,data_source,load_time,effective_time,discontinue_time,active_flag
0,203681621USAPGD,20368,1621,12544,1254403,32544,USA,14082,1408202,34082,PGD,bird,2024-01-26 22:23:44.946037+00:00,2024-02-11 21:39:45.621337+00:00,<NA>,True
1,203681931USAVPS,20368,1931,12544,1254403,32544,USA,15624,1562404,31504,VPS,bird,2024-01-26 22:23:44.946037+00:00,2024-02-11 21:39:45.621337+00:00,<NA>,True
2,20368819USAPIE,20368,819,12544,1254403,32544,USA,14112,1411206,33195,PIE,bird,2024-01-26 22:23:44.946037+00:00,2024-02-11 21:39:45.621337+00:00,<NA>,True
3,20368639USASFB,20368,639,12544,1254403,32544,USA,14761,1476106,34761,SFB,bird,2024-01-26 22:23:44.946037+00:00,2024-02-11 21:39:45.621337+00:00,<NA>,True
4,203682139USASFB,20368,2139,12544,1254403,32544,USA,14761,1476106,34761,SFB,bird,2024-01-26 22:23:44.946037+00:00,2024-02-11 21:39:45.621337+00:00,<NA>,True


In [ ]:
%%bigquery
alter table airline_csp.Flight add primary key (fl_num, effective_time) not enforced

Query is running:   0%|          |

""


# Flight_History

In [ ]:
%%bigquery
create or replace table airline_csp.Flight_History
  as select *, current_timestamp() as effective_time, null as discontinue_time, true as active_flag
  from airline_stg.Flight_History

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select * from airline_csp.Flight_History
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,fl_date,fl_num,tail_num,crs_dep_time,dep_time,dep_delay,dep_delay_new,arr_time,arr_delay,arr_delay_new,...,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,data_source,load_time,effective_time,discontinue_time,active_flag
0,2018/8/3,203983441CVGJFK,N826AE,1241,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,bird,2024-01-26 22:23:44.946037+00:00,2024-02-11 21:40:08.739549+00:00,<NA>,True
1,2018/8/13,20436875TTNPBI,None,1053,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,bird,2024-01-26 22:23:44.946037+00:00,2024-02-11 21:40:08.739549+00:00,<NA>,True
2,2018/8/8,203662898DFWBTR,N611QX,1614,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,bird,2024-01-26 22:23:44.946037+00:00,2024-02-11 21:40:08.739549+00:00,<NA>,True
3,2018/8/8,203983559MEMORD,N538EG,551,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,bird,2024-01-26 22:23:44.946037+00:00,2024-02-11 21:40:08.739549+00:00,<NA>,True
4,2018/8/7,203045393ORDOKC,N143SY,1932,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,bird,2024-01-26 22:23:44.946037+00:00,2024-02-11 21:40:08.739549+00:00,<NA>,True


In [ ]:
%%bigquery
alter table airline_csp.Flight_History add primary key (fl_date, fl_num, effective_time) not enforced

Query is running:   0%|          |

""


# Meal_Service

In [ ]:
%%bigquery
create or replace table airline_csp.Meal_Service
  as select *, current_timestamp() as effective_time, null as discontinue_time, true as active_flag
  from airline_stg.Meal_Service

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select * from airline_csp.Meal_Service
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,fl_num,meal_id,airline_id,service_category,data_source,load_time,effective_time,discontinue_time,active_flag
0,19930751PDXSEA,52998,19930,None,bird_mealdb,2024-02-04 16:07:07.591797+00:00,2024-02-11 21:40:36.364982+00:00,<NA>,True
1,19930169DLGANC,52998,19930,None,bird_mealdb,2024-02-04 16:07:07.591797+00:00,2024-02-11 21:40:36.364982+00:00,<NA>,True
2,199301871DALLAX,52998,19930,None,bird_mealdb,2024-02-04 16:07:07.591797+00:00,2024-02-11 21:40:36.364982+00:00,<NA>,True
3,199301897LASSFO,52998,19930,None,bird_mealdb,2024-02-04 16:07:07.591797+00:00,2024-02-11 21:40:36.364982+00:00,<NA>,True
4,199301320SEALAS,52998,19930,None,bird_mealdb,2024-02-04 16:07:07.591797+00:00,2024-02-11 21:40:36.364982+00:00,<NA>,True


In [ ]:
%%bigquery
alter table airline_csp.Meal_Service add primary key (fl_num, meal_id, effective_time) not enforced

Query is running:   0%|          |

""


# Snack_Service

In [ ]:
%%bigquery
create or replace table airline_csp.Snack_Service
  as select *, current_timestamp() as effective_time, null as discontinue_time, true as active_flag
  from airline_stg.Snack_Service

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select * from airline_csp.Snack_Service
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,fl_num,snack_id,airline_id,service_category,data_source,load_time,effective_time,discontinue_time,active_flag
0,203043376SFOABQ,9300000482,20304,None,bird_open_food_facts,2024-02-04 16:15:22.761151+00:00,2024-02-11 21:40:56.127028+00:00,<NA>,True
1,203045555SFOACV,9300000482,20304,None,bird_open_food_facts,2024-02-04 16:15:22.761151+00:00,2024-02-11 21:40:56.127028+00:00,<NA>,True
2,203043579ATLAEX,9300000482,20304,None,bird_open_food_facts,2024-02-04 16:15:22.761151+00:00,2024-02-11 21:40:56.127028+00:00,<NA>,True
3,203044466CHAATL,9300000482,20304,None,bird_open_food_facts,2024-02-04 16:15:22.761151+00:00,2024-02-11 21:40:56.127028+00:00,<NA>,True
4,203044472DHNATL,9300000482,20304,None,bird_open_food_facts,2024-02-04 16:15:22.761151+00:00,2024-02-11 21:40:56.127028+00:00,<NA>,True


In [ ]:
%%bigquery
alter table airline_csp.Snack_Service add primary key (fl_num, snack_id, effective_time) not enforced

Query is running:   0%|          |

""


# Meal

In [ ]:
%%bigquery
create or replace table airline_csp.Meal
  as select *, current_timestamp() as effective_time, null as discontinue_time, true as active_flag
  from airline_stg.Meal

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select * from airline_csp.Meal
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,meal_id,meal_name,meal_image,cat_name,tags,area,ingredient1,ingredient2,ingredient3,ingredient4,ingredient5,source,youtube,data_source,load_time,effective_time,discontinue_time,active_flag
0,52827,Massaman Beef curry,https://www.themealdb.com/images/media/meals/t...,Beef,Curry,Thai,Peanuts,Coconut cream,Massaman curry paste,Beef,Potatoes,https://www.bbcgoodfood.com/recipes/420631/bee...,https://www.youtube.com/watch?v=mVxgZSCU9_g,mealdb,2024-01-26 22:24:02.252144+00:00,2024-02-11 21:41:13.619434+00:00,<NA>,True
1,52979,Bitterballen (Dutch meatballs),https://www.themealdb.com/images/media/meals/l...,Beef,"DinnerParty,HangoverFood,Alcoholic",Dutch,Butter,Flour,Beef Stock,Onion,Parsley,https://www.holland.com/global/tourism/informa...,https://www.youtube.com/watch?v=q8AKfYUtDuM,mealdb,2024-01-26 22:24:02.252144+00:00,2024-02-11 21:41:13.619434+00:00,<NA>,True
2,53006,Moussaka,https://www.themealdb.com/images/media/meals/c...,Beef,None,Greek,Beef,Aubergine,Greek Yogurt,Egg,Parmesan,https://www.bbcgoodfood.com/recipes/must-make-...,https://www.youtube.com/watch?v=8U_29i9Qp5U,mealdb,2024-01-26 22:24:02.252144+00:00,2024-02-11 21:41:13.619434+00:00,<NA>,True
3,53000,Vegetable Shepherd's Pie,https://www.themealdb.com/images/media/meals/w...,Beef,Alcoholic,Irish,Potatoes,Small Potatoes,Salted Butter,Mushrooms,Brown Lentils,None,None,mealdb,2024-01-26 22:24:02.252144+00:00,2024-02-11 21:41:13.619434+00:00,<NA>,True
4,52998,Corned Beef and Cabbage,https://www.themealdb.com/images/media/meals/x...,Beef,None,Irish,Beef Brisket,Small Potatoes,Carrots,Cabbage,None,None,None,mealdb,2024-01-26 22:24:02.252144+00:00,2024-02-11 21:41:13.619434+00:00,<NA>,True


In [ ]:
%%bigquery
alter table airline_csp.Meal add primary key (meal_id, effective_time) not enforced

Query is running:   0%|          |

""


# Snack

In [ ]:
%%bigquery
create or replace table airline_csp.Snack
  as select *, current_timestamp() as effective_time, null as discontinue_time, true as active_flag
  from airline_stg.Snack

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select * from airline_csp.Snack
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,snack_id,url,product_name,brands,categories,countries_en,ingredients_text,image_url,created_datetime,last_modified_datetime,data_source,load_time,effective_time,discontinue_time,active_flag
0,3245412567292,http://world-en.openfoodfacts.org/product/3245...,"Chips de crevettes, recette vietnamienne",Carrefour,"Aliments et boissons à base de végétaux, Alime...",None,None,https://images.openfoodfacts.org/images/produc...,2017-03-12 17:32:18+00:00,2022-12-21 10:56:17+00:00,open_food_facts,2024-02-03 19:40:10.880358+00:00,2024-02-11 21:41:38.103172+00:00,<NA>,True
1,8377050,http://world-en.openfoodfacts.org/product/0837...,Barbecue Potato Chips,UTZ,Snacks,None,None,https://images.openfoodfacts.org/images/produc...,2022-02-21 23:43:52+00:00,2022-10-04 09:20:53+00:00,open_food_facts,2024-02-03 19:40:10.880358+00:00,2024-02-11 21:41:38.103172+00:00,<NA>,True
2,30000576731,http://world-en.openfoodfacts.org/product/0030...,Rice Thins Cocoa Crunch,Quaker Oats,Snacks,None,None,https://images.openfoodfacts.org/images/produc...,2023-11-25 13:07:16+00:00,2023-11-25 13:23:49+00:00,open_food_facts,2024-02-03 19:40:10.880358+00:00,2024-02-11 21:41:38.103172+00:00,<NA>,True
3,860001166743,http://world-en.openfoodfacts.org/product/0860...,Sunflower Fig bar,WellBean,Snacks,None,None,https://images.openfoodfacts.org/images/produc...,2023-12-10 17:23:07+00:00,2023-12-10 17:36:41+00:00,open_food_facts,2024-02-03 19:40:10.880358+00:00,2024-02-11 21:41:38.103172+00:00,<NA>,True
4,8906125471782,http://world-en.openfoodfacts.org/product/8906...,Finger Stix,Star,Snacks,None,"Corn Flour, Potato Powder, Refined Palmolein O...",https://images.openfoodfacts.org/images/produc...,2023-12-18 12:08:28+00:00,2023-12-20 01:14:40+00:00,open_food_facts,2024-02-03 19:40:10.880358+00:00,2024-02-11 21:41:38.103172+00:00,<NA>,True


In [ ]:
%%bigquery
alter table airline_csp.Snack add primary key (snack_id, effective_time) not enforced

Query is running:   0%|          |

""


# Foreign Keys

Due to the addition of `effective_date`, we can not create foreign keys on our tables in the consumption layer. That is why it is very important to verify FKs in the staging layer.  

In [ ]:
%%bigquery
alter table airline_csp.Flight add foreign key (origin)
  references airline_csp.Airport (iata) not enforced

Executing query with job ID: fc8c057f-ecec-4327-99ac-7a9bb1238ba3
Query executing: 0.43s


ERROR:
 400 Foreign Key constraint references 1 column, but the referenced table airline_csp.Airport has 2 primary keys

Location: US
Job ID: fc8c057f-ecec-4327-99ac-7a9bb1238ba3

